In [8]:
from anthropic import Anthropic
from anthropic.types import Message
import base64
from PIL import Image
import io

from dotenv import load_dotenv
import os   
load_dotenv()

import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

try:
    client = Anthropic(
        api_key=os.getenv("ANTHROPIC_API_KEY"),
    )
    logger.info("Anthropic client created successfully.")
except Exception as e:
    logger.error(f"Failed to create Anthropic client: {e}")
    raise

2025-08-10 20:50:26 - __main__ - INFO - Anthropic client created successfully.


In [9]:
MODEL="claude-3-7-sonnet-20250219"
TEMPERATURE=1.0

In [10]:
def add_user_message(messages, message_content):
    if isinstance(message_content, list):
        user_message = {
            "role": "user",
            "content": message_content
        }
    else:
        user_message = {
            "role": "user",
            "content": [{"type": "text", "text": str(message_content)}]
        }

    messages.append(user_message)


def add_assistant_message(messages, message_content):
    if isinstance(message_content, Message):
        assistant_message = {
            "role": "assistant",
            "content": message_content
        }
    else:
        assistant_message = {
            "role": "assistant",
            "content": [{"type": "text", "text": message_content}]
        }

    messages.append(assistant_message)

def chat(messages, model=MODEL, temperature=TEMPERATURE, system=None, stop_sequences=None, thinking=False, thinking_budget=1024, tools=None):
    try:
        params = {
            "model": model,
            "max_tokens": 4000,
            "messages": messages,
            "temperature": temperature,
            "stop_sequences": stop_sequences,
        }
        
        if thinking:
            params["thinking"] = {
                "type": "enabled",
                "budget_tokens": thinking_budget
            }
        
        if tools:
            params["tools"] = tools

        if system:
            params["system"] = system

        return client.messages.create(**params)
    
    except Exception as e:
        logger.error(f"Chat failed: {e}")
        raise e
    
def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

In [11]:
RISK_ASSESSMENT_PROMPT = """
Analyze the attached satellite image of a property with these specific steps:

1. Residence identification: Locate the primary residence on the property by looking for:
   - The largest roofed structure 
   - Typical residential features (driveway connection, regular geometry)
   - Distinction from other structures (garages, sheds, pools)
   Describe the residence's location relative to property boundaries and other features.

2. Tree overhang analysis: Examine all trees near the primary residence:
   - Identify any trees whose canopy extends directly over any portion of the roof
   - Estimate the percentage of roof covered by overhanging branches (0-25%, 25-50%, 50-75%, 75-100%)
   - Note particularly dense areas of overhang

3. Fire risk assessment: For any overhanging trees, evaluate:
   - Potential wildfire vulnerability (ember catch points, continuous fuel paths to structure)
   - Proximity to chimneys, vents, or other roof openings if visible
   - Areas where branches create a "bridge" between wildland vegetation and the structure
   
4. Defensible space identification: Assess the property's overall vegetative structure:
   - Identify if trees connect to form a continuous canopy over or near the home
   - Note any obvious fuel ladders (vegetation that can carry fire from ground to tree to roof)

5. Fire risk rating: Based on your analysis, assign a Fire Risk Rating from 1-4:
   - Rating 1 (Low Risk): No tree branches overhanging the roof, good defensible space around the structure
   - Rating 2 (Moderate Risk): Minimal overhang (<25% of roof), some separation between tree canopies
   - Rating 3 (High Risk): Significant overhang (25-50% of roof), connected tree canopies, multiple points of vulnerability
   - Rating 4 (Severe Risk): Extensive overhang (>50% of roof), dense vegetation against structure, numerous ember catch points, limited defensible space

For each item above (1-5), write one sentence summarizing your findings, with your final response being the numeric Fire Risk Rating (1-4) with a brief justification.
"""

In [12]:
def compress_image(img_file, max_size=(1024, 1024), quality=85):
    """Compress image to reduce file size"""
    try:
        with Image.open(img_file) as img:
            # Convert to RGB if necessary
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Resize if too large
            img.thumbnail(max_size, Image.Resampling.LANCZOS)
            
            # Save to bytes
            buffer = io.BytesIO()
            img.save(buffer, format='JPEG', quality=quality)
            return buffer.getvalue()
    except Exception as e:
        logger.error(f"Failed to compress image: {e}")
        raise e
    
def upload_image(img_file, messages):
    try:
        compressed_image = compress_image(img_file)
        image_bytes = base64.standard_b64encode(compressed_image).decode("utf-8")

        add_user_message(messages, [
            # Image Block
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/jpeg",
                    "data": image_bytes,
                }
            },
            # Text Block
            {
                "type": "text",
                "text": "What do you see in this image?"
            }
        ])
        logger.info("Image uploaded successfully.")
    except Exception as e:
        logger.error(f"Failed to upload image: {e}")
        raise e

In [13]:
def main(img_file):
    messages = []
    
    try:
        # Print user input
        print("=" * 50)
        print("USER:")
        print("=" * 50)
        print(f"Image file: {img_file}")
        print("Analyzing image for fire risk...")
        print()
        
        upload_image(img_file, messages) 
        add_user_message(messages, RISK_ASSESSMENT_PROMPT)

        logger.info("Starting chat with model...")
        results = chat(messages, model=MODEL, temperature=TEMPERATURE, thinking=True, thinking_budget=1024)
        logger.info("Chat completed successfully.")
        
        print("=" * 50)
        print("ASSISTANT:")
        print("=" * 50)
        print(text_from_message(results))
        print()
        
        return results

    except Exception as e:
        logger.error(f"Error in main function: {e}")
        raise e

In [14]:
results = main("assets/images/prop1.png")

USER:
Image file: assets/images/prop1.png
Analyzing image for fire risk...



2025-08-10 20:50:26 - __main__ - INFO - Image uploaded successfully.
2025-08-10 20:50:26 - __main__ - INFO - Starting chat with model...
2025-08-10 20:50:40 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-08-10 20:50:40 - __main__ - INFO - Chat completed successfully.


ASSISTANT:
# Property Fire Risk Analysis

Looking at this aerial image, I can see a primary residence situated in the center of the property, surrounded by dense tree coverage. The structure appears to have a grayish/brown roof and is partially obscured by surrounding vegetation.

The trees around the residence show significant canopy coverage extending over the roof, with approximately 25-50% of the roof area affected by overhanging branches, particularly on what appears to be the northern and eastern sides of the structure.

From a fire risk perspective, these overhanging branches create numerous potential ember catch points, with the dense vegetation forming continuous fuel paths directly to the structure, which could allow fire to spread quickly from the surrounding wildland to the building.

The property shows limited defensible space, with trees forming an almost continuous canopy connecting the surrounding woodland directly to the residence, creating multiple fuel ladders that c

In [15]:
print(results)

Message(id='msg_011WgotDRXk1TRXtx1TAKqK3', content=[ThinkingBlock(signature='ErUBCkYIBhgCIkBxl3baEE20hOUk9vpfnktgShHdvJ7syFcZG3/IEEJrKOiRvO6FCrihh6uRxWIpaCfuS5EPGuRvAU2+cJyBf2xkEgy/o5J3cAxsHu6s8RwaDDXwW4q/97/gDE5xaSIwjTFVcIohz4lVDmYaxifGxHO2CHgFaXGdW7MhzcF7AGX+hxh6EavsyBM8c8eBbodKKh2rhjTnD4QGx0qbSQX92vPfBtKSw5z7jrxqMf/XfRgC', thinking="Let me analyze this satellite image of a property, following the requested steps:\n\n1. Residence identification:\nI can see what appears to be a structure in the middle of the image, partially obscured by trees. It has a brownish/grayish roof and seems to be a house or similar building. It's situated in a clearing surrounded by trees and green space.\n\n2. Tree overhang analysis:\nThere appear to be multiple trees surrounding the structure, and some of them seem to have canopies that extend over portions of the roof. Based on the image, I'd estimate that approximately 25-50% of the roof may be covered by overhanging branches. The trees appear quite dens

In [16]:
print(f"thinking signature: \n{results.content[0].signature}")
print(f"\nthinking content: \n{results.content[0].thinking}")

thinking signature: 
ErUBCkYIBhgCIkBxl3baEE20hOUk9vpfnktgShHdvJ7syFcZG3/IEEJrKOiRvO6FCrihh6uRxWIpaCfuS5EPGuRvAU2+cJyBf2xkEgy/o5J3cAxsHu6s8RwaDDXwW4q/97/gDE5xaSIwjTFVcIohz4lVDmYaxifGxHO2CHgFaXGdW7MhzcF7AGX+hxh6EavsyBM8c8eBbodKKh2rhjTnD4QGx0qbSQX92vPfBtKSw5z7jrxqMf/XfRgC

thinking content: 
Let me analyze this satellite image of a property, following the requested steps:

1. Residence identification:
I can see what appears to be a structure in the middle of the image, partially obscured by trees. It has a brownish/grayish roof and seems to be a house or similar building. It's situated in a clearing surrounded by trees and green space.

2. Tree overhang analysis:
There appear to be multiple trees surrounding the structure, and some of them seem to have canopies that extend over portions of the roof. Based on the image, I'd estimate that approximately 25-50% of the roof may be covered by overhanging branches. The trees appear quite dense and close to the structure.

3. Fire risk assessment